<a href="https://colab.research.google.com/github/tada20001/NLP_2023/blob/main/CH9_07_%EC%9E%90%EB%AA%A8%EB%8B%A8%EC%9C%84_%ED%95%9C%EA%B5%AD%EC%96%B4_FastText_%ED%95%99%EC%8A%B5%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Colab에 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 115 (delta 11), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (115/115), 1.27 MiB | 5.22 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 KB 12.0 MB/s eta 0:00:00
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2023-03-13 05:38:52--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::3403:4be7, 2406:da00:ff00::

In [3]:
# 한글 자모 단위 처리 패키지 설치
!pip install hgtk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
# fasttext 설치
!git clone https://github.com/facebookresearch/fastText.git
%cd fastText
!make
!pip install .

Cloning into 'fastText'...
remote: Enumerating objects: 3930, done.
remote: Counting objects: 100% (943/943), done.
remote: Compressing objects: 100% (137/137), done.
remote: Total 3930 (delta 854), reused 806 (delta 806), pack-reused 2987
Receiving objects: 100% (3930/3930), 8.24 MiB | 17.22 MiB/s, done.
Resolving deltas: 100% (2505/2505), done.
/content/Mecab-ko-for-Google-Colab/fastText
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/args.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/autotune.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/matrix.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/dictionary.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/loss.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/productquantizer.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/densematrix.cc
c++ -pthr

### 1. 데이터 로드

In [5]:
import re
import pandas as pd
import urllib.request
from tqdm import tqdm
import hgtk
from konlpy.tag import Mecab

In [6]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt", filename="ratings_total.txt")


('ratings_total.txt', <http.client.HTTPMessage at 0x7f413cc44f70>)

In [7]:
total_data = pd.read_table('ratings_total.txt', names=['ratings', 'reviews'])
total_data.tail()

,ratings,reviews
199995,2,장마라그런가!!! 달지않아요
199996,5,다이슨 케이스 구매했어요 다이슨 슈퍼소닉 드라이기 케이스 구매했어요가격 괜찮고 배송...
199997,5,로드샾에서 사는것보다 세배 저렴하네요 ㅜㅜ 자주이용할께요
199998,5,넘이쁘고 쎄련되보이네요~
199999,5,아직 사용해보지도않았고 다른 제품을 써본적이없어서 잘 모르겠지만 ㅎㅎ 배송은 빨랐습니다


### 2. hgtk 튜토리얼

In [8]:
# 한글인지 체크
hgtk.checker.is_hangul('ㄱ')

True

In [9]:
hgtk.checker.is_hangul('28')

False

In [10]:
# 음절을 초성, 중성, 종성으로 분해
hgtk.letter.decompose('남')

('ㄴ', 'ㅏ', 'ㅁ')

In [11]:
# 초성, 중성 결합
hgtk.letter.compose('ㄴ', 'ㅏ')

'나'

In [12]:
hgtk.letter.compose('ㄴ', 'ㅏ', 'ㅁ')

'남'

In [13]:
# 한글이 아닌 입력에 대해서는 에러 발생
hgtk.letter.decompose('1')

NotHangulException: ignored

In [14]:
# 결합할 수 없는 상황에서도 에러 발생
hgtk.letter.compose('ㄴ', 'ㅁ', 'ㅁ')

NotHangulException: ignored

### 3. 데이터 전처리

In [15]:
def word_to_jamo(token):
  def to_special_token(jamo):
    if not jamo:
      return '-'
    else:
      return jamo
    
  decomposed_token = ''
  for char in token:
    try:
      cho, jung, jong = hgtk.letter.decompose(char)
      cho = to_special_token(cho)
      jung = to_special_token(jung)
      jong = to_special_token(jong)
      decomposed_token = decomposed_token + cho + jung + jong
    except Exception as exception:
      if type(exception).__name__ == 'NotHangulException':
        decomposed_token += char

  return decomposed_token

In [16]:
word_to_jamo('남동생')

'ㄴㅏㅁㄷㅗㅇㅅㅐㅇ'

In [17]:
word_to_jamo('2012년')

'2012ㄴㅕㄴ'

In [18]:
mecab = Mecab()
print(mecab.morphs('선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다.'))

['선물', '용', '으로', '빨리', '받', '아서', '전달', '했어야', '하', '는', '상품', '이', '었', '는데', '머그', '컵', '만', '와서', '당황', '했', '습니다', '.']


In [19]:
def tokenize_by_jamo(sentence):
  return [word_to_jamo(token) for token in mecab.morphs(sentence)]

In [20]:
print(tokenize_by_jamo('선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다.'))

['ㅅㅓㄴㅁㅜㄹ', 'ㅇㅛㅇ', 'ㅇㅡ-ㄹㅗ-', 'ㅃㅏㄹㄹㅣ-', 'ㅂㅏㄷ', 'ㅇㅏ-ㅅㅓ-', 'ㅈㅓㄴㄷㅏㄹ', 'ㅎㅐㅆㅇㅓ-ㅇㅑ-', 'ㅎㅏ-', 'ㄴㅡㄴ', 'ㅅㅏㅇㅍㅜㅁ', 'ㅇㅣ-', 'ㅇㅓㅆ', 'ㄴㅡㄴㄷㅔ-', 'ㅁㅓ-ㄱㅡ-', 'ㅋㅓㅂ', 'ㅁㅏㄴ', 'ㅇㅘ-ㅅㅓ-', 'ㄷㅏㅇㅎㅘㅇ', 'ㅎㅐㅆ', 'ㅅㅡㅂㄴㅣ-ㄷㅏ-', '.']


In [21]:
tokenized_data = []

for sample in tqdm(total_data['reviews'].to_list()):
  tokenized_data.append(tokenize_by_jamo(sample))

100%|██████████| 200000/200000 [01:13<00:00, 2737.61it/s]


In [22]:
len(tokenized_data)

200000

In [23]:
tokenized_data[0]  # 4개 토큰에 대한 자모단위 분리

['ㅂㅐ-ㄱㅗㅇ', 'ㅃㅏ-ㄹㅡ-', 'ㄱㅗ-', 'ㄱㅜㅅ']

In [24]:
total_data['reviews'][0]

'배공빠르고 굿'

In [25]:
# 자모에서 단어로 합치기기
def jamo_to_word(jamo_sequence):
  tokenized_jamo = []
  index = 0

  # 1. jamo 초기 입력 
  while index < len(jamo_sequence):
    if not hgtk.checker.is_hangul(jamo_sequence[index]):  # 글자 하나씩 한글인지 아닌지 체크
      tokenized_jamo.append(jamo_sequence[index])
      index += 1
    ## 문자가 정상적인 자모라면 초성, 중성, 종성을 하나의 토큰으로 간주
    else:
      tokenized_jamo.append(jamo_sequence[index:index+3])
      index += 3
  
  # 2. 자모 단위 토큰화 완료
  # tokenized_jamo : ['ㄴㅏㅁ', 'ㄷㅗㅇ', 'ㅅㅐㅇ']
  word = ''
  try:
    for jamo in tokenized_jamo:
      # 초성, 중성, 중성 묶으로 추정되는 경우
      if len(jamo) == 3:
        if jamo[2] == '-':
          word = word + hgtk.letter.compose(jamo[0], jamo[1])
        else:
          word = word + hgtk.letter.compose(jamo[0], jamo[1], jamo[2])
      else:
        word = word + jamo

  except Exception as exception:
    if type(exception).__name__ == 'NotHangulException':
      return jamo_sequence

  # 3. 단어로 복원 
  return word

In [26]:
jamo_to_word('ㄴㅏㅁㄷㅗㅇㅅㅐㅇ')

'남동생'

### 4. FastText

In [27]:
import fasttext

In [28]:
with open('tokenized_data.txt', 'w') as out:
  for line in tqdm(tokenized_data, unit=' line'):
    out.write(' '.join(line) + '\n')

100%|██████████| 200000/200000 [00:00<00:00, 228093.18 line/s]


In [29]:
model = fasttext.train_unsupervised('tokenized_data.txt', model='cbow')

In [ ]:
model.save_model('fasttext.bin')  # 모델 저장
model = fasttext.load_model('fasttext.bin')

In [30]:
model[word_to_jamo('남동생')]

array([-4.2318109e-01, -9.1394566e-02,  6.6527560e-02,  2.8613076e-01,
       -4.0564996e-01, -2.6235077e-01,  8.4429610e-01,  5.0584716e-01,
       -6.0741201e-02,  7.6125443e-02, -2.4852820e-01, -1.9942357e-01,
        5.0343859e-01, -1.4968413e-01,  2.3070360e-02, -4.7695845e-01,
        9.3166995e-01,  4.5898706e-01, -1.7003957e-02,  2.2412080e-01,
        5.8578465e-02,  2.4508233e-01,  4.5507513e-02, -1.1122880e-01,
        1.1093882e+00, -2.2732276e-01,  5.6505400e-01,  1.1018128e+00,
       -2.3074827e-01,  7.3671079e-01,  6.6943914e-01, -4.1340268e-01,
        6.5859085e-01, -1.4125316e+00, -2.5944665e-01,  7.9829924e-02,
        8.5497814e-01,  5.2773821e-01,  2.8628391e-01,  1.7737828e-01,
        2.9157144e-01, -8.2901156e-01, -7.7729940e-01, -2.5763926e-01,
       -7.3712325e-01, -7.6144409e-01, -1.1119006e+00, -4.5516741e-01,
       -5.9126055e-01,  1.3821359e-01,  6.9247818e-01,  8.9022383e-02,
       -3.8963851e-01,  1.7265147e-01, -3.6614782e-01, -3.2631320e-01,
      

In [31]:
model.get_nearest_neighbors(word_to_jamo('남동생'))

[(0.8919562101364136, 'ㄷㅗㅇㅅㅐㅇ'),
 (0.8326149582862854, 'ㄴㅏㅁㅊㅣㄴ'),
 (0.7556453347206116, 'ㅈㅜㅇㅎㅏㄱㅅㅐㅇ'),
 (0.7532448172569275, 'ㅊㅣㄴㄱㅜ-'),
 (0.7416449785232544, 'ㅅㅐㅇㅇㅣㄹ'),
 (0.7385997772216797, 'ㄴㅏㅁㅇㅏ-'),
 (0.7238298058509827, 'ㅎㅏㄱㅅㅐㅇ'),
 (0.7128468751907349, 'ㅈㅗ-ㅋㅏ-'),
 (0.7124418616294861, 'ㅊㅗ-ㄷㅡㅇㅎㅏㄱㅅㅐㅇ'),
 (0.7089083194732666, 'ㄴㅏㄴㅅㅐㅇ')]

In [41]:
def transform(word):
  word_sequence = model.get_nearest_neighbors(word_to_jamo(word), k=10)
  return [(jamo_to_word(word), similarity) for similarity, word in word_sequence]

In [42]:
transform('남동생')

[('동생', 0.8919562101364136),
 ('남친', 0.8326149582862854),
 ('중학생', 0.7556453347206116),
 ('친구', 0.7532448172569275),
 ('생일', 0.7416449785232544),
 ('남아', 0.7385997772216797),
 ('학생', 0.7238298058509827),
 ('조카', 0.7128468751907349),
 ('초등학생', 0.7124418616294861),
 ('난생', 0.7089083194732666)]

In [43]:
transform('남동쉥')

[('남동생', 0.8978480100631714),
 ('남친', 0.8136335015296936),
 ('남매', 0.7744753956794739),
 ('남짓', 0.7336941957473755),
 ('남긴', 0.7175815105438232),
 ('남김', 0.7094102501869202),
 ('남겼', 0.7080584764480591),
 ('남아', 0.7079014778137207),
 ('남길', 0.6984111070632935),
 ('남녀', 0.695271372795105)]

In [44]:
transform('남동쉥ㅋ')

[('남동생', 0.8319035172462463),
 ('남친', 0.7404624223709106),
 ('남아', 0.6615323424339294),
 ('남짓', 0.6601576805114746),
 ('남매', 0.6472423672676086),
 ('남녀', 0.6420586705207825),
 ('남여', 0.6406536102294922),
 ('동생', 0.6306872963905334),
 ('남김', 0.6287914514541626),
 ('남길', 0.6162610054016113)]

In [45]:
transform('난동쉥')

[('난다', 0.8086841106414795),
 ('납땜', 0.7974720597267151),
 ('낚', 0.7524359822273254),
 ('난생', 0.7509694695472717),
 ('낳', 0.7509130835533142),
 ('난닝', 0.7503401041030884),
 ('난방', 0.7402766942977905),
 ('낟', 0.739186704158783),
 ('낡', 0.7313180565834045),
 ('난국', 0.7280634641647339)]

In [46]:
transform('난동생')

[('남동생', 0.8612213134765625),
 ('난생', 0.8565025329589844),
 ('동생', 0.7903454303741455),
 ('남아', 0.7870966196060181),
 ('중학생', 0.7495378851890564),
 ('남친', 0.7430869340896606),
 ('고등학생', 0.7274073362350464),
 ('남편', 0.7241329550743103),
 ('남매', 0.7170072793960571),
 ('초등학생', 0.7100576162338257)]

In [47]:
transform('고품질')

[('품질', 0.850900411605835),
 ('음질', 0.8198748230934143),
 ('땜질', 0.7482479810714722),
 ('찜질', 0.6913276314735413),
 ('군것질', 0.6653683185577393),
 ('고감', 0.6636772155761719),
 ('사포질', 0.6631086468696594),
 ('성질', 0.6547363996505737),
 ('다림질', 0.6520350575447083),
 ('재품', 0.650108277797699)]

In [48]:
transform('노품질')

[('고품질', 0.8959778547286987),
 ('품질', 0.861327052116394),
 ('음질', 0.7958589196205139),
 ('땜질', 0.7383509874343872),
 ('찜질', 0.6871261596679688),
 ('퀄리티', 0.6663155555725098),
 ('화질', 0.6601850986480713),
 ('사포질', 0.6501261591911316),
 ('성질', 0.6366932392120361),
 ('소모품', 0.6337293386459351)]

In [49]:
transform('제품')

[('반제품', 0.8980190753936768),
 ('완제품', 0.8666797280311584),
 ('재품', 0.8395289182662964),
 ('상품', 0.8249848484992981),
 ('타제품', 0.8207210302352905),
 ('제풍', 0.8058335781097412),
 ('화학제품', 0.7825376391410828),
 ('중품', 0.7738685607910156),
 ('최상품', 0.7691351175308228),
 ('명품', 0.7670924663543701)]